In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import time
import pandas as pd

# Function to create model
def create_model(model_type, sequence_length, vocab_size):
    model = Sequential()
    if model_type == "RNN":
        model.add(tf.keras.layers.SimpleRNN(128, input_shape=(sequence_length, vocab_size)))
    elif model_type == "LSTM":
        model.add(LSTM(128, input_shape=(sequence_length, vocab_size)))
    elif model_type == "GRU":
        model.add(GRU(128, input_shape=(sequence_length, vocab_size)))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Function to train model and measure execution time
def train_model(model, X_train, y_train, validation_split):
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, validation_split=validation_split, batch_size=16, shuffle=False, callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=0)])
    end_time = time.time()
    execution_time = end_time - start_time
    return history, execution_time

# Function to evaluate model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    return loss, accuracy

# Function to generate sequences
def generate_sequences(text, max_sequence_length, step):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - max_sequence_length, step):
        sequences.append(text[i:i + max_sequence_length])
        next_chars.append(text[i + max_sequence_length])
    return sequences, next_chars

# Function to vectorize sequences
def vectorize_sequences(sequences, next_chars, char_to_idx, vocab_size):
    X = np.zeros((len(sequences), max_sequence_length, vocab_size), dtype=bool)
    y = np.zeros((len(sequences), vocab_size), dtype=bool)
    for i, sequence in enumerate(sequences):
        for t, char in enumerate(sequence):
            X[i, t, char_to_idx[char]] = 1
        y[i, char_to_idx[next_chars[i]]] = 1
    return X, y

# Define the text data and preprocessing steps
text = "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."
vocab = sorted(set(text))
vocab_size = len(vocab)
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
max_sequence_lengths = [10, 20, 30]
step = 1
validation_split = 0.2

# Report variables
report = []

# Loop through different sequence lengths
for max_sequence_length in max_sequence_lengths:
    sequences, next_chars = generate_sequences(text, max_sequence_length, step)
    X, y = vectorize_sequences(sequences, next_chars, char_to_idx, vocab_size)

    # Models
    models = [("RNN", create_model("RNN", max_sequence_length, vocab_size)),
              ("LSTM", create_model("LSTM", max_sequence_length, vocab_size)),
              ("GRU", create_model("GRU", max_sequence_length, vocab_size))]

    # Training and evaluation
    for model_type, model in models:
        history, execution_time = train_model(model, X, y, validation_split)
        loss, accuracy = evaluate_model(model, X, y)
        report.append((model_type, max_sequence_length, history.history['loss'][-1], history.history['val_accuracy'][-1], execution_time, model.count_params()))

# Create DataFrame
df = pd.DataFrame(report, columns=["Model Type", "Sequence Length", "Training Loss", "Validation Accuracy", "Execution Time", "Model Complexity"])

Epoch 1/100
119/119 [==============================] - 2s 9ms/step - loss: 3.0906 - accuracy: 0.1700 - val_loss: 3.0081 - val_accuracy: 0.1702
Epoch 2/100
119/119 [==============================] - 1s 6ms/step - loss: 2.6958 - accuracy: 0.2721 - val_loss: 2.8934 - val_accuracy: 0.2143
Epoch 3/100
119/119 [==============================] - 1s 6ms/step - loss: 2.4799 - accuracy: 0.3353 - val_loss: 2.8190 - val_accuracy: 0.2311
Epoch 4/100
119/119 [==============================] - 1s 7ms/step - loss: 2.3012 - accuracy: 0.3742 - val_loss: 2.7653 - val_accuracy: 0.2458
Epoch 5/100
119/119 [==============================] - 1s 6ms/step - loss: 2.1468 - accuracy: 0.4126 - val_loss: 2.7244 - val_accuracy: 0.2689
Epoch 6/100
119/119 [==============================] - 1s 7ms/step - loss: 2.0085 - accuracy: 0.4453 - val_loss: 2.6952 - val_accuracy: 0.2857
Epoch 7/100
119/119 [==============================] - 1s 7ms/step - loss: 1.8803 - accuracy: 0.4726 - val_loss: 2.6795 - val_accuracy: 0.3025

In [ ]:
df

,Model Type,Sequence Length,Training Loss,Validation Accuracy,Execution Time,Model Complexity
0,RNN,10,0.719714,0.302521,16.078991,27820
1,LSTM,10,1.120431,0.321429,37.742240,94252
2,GRU,10,0.704056,0.386555,34.934811,72492
3,RNN,20,1.021968,0.251055,22.568749,27820
4,LSTM,20,1.227047,0.350211,59.930146,94252
5,GRU,20,0.702583,0.398734,52.788986,72492
6,RNN,30,1.257410,0.315678,23.488577,27820
7,LSTM,30,1.377033,0.341102,82.479621,94252
8,GRU,30,0.932051,0.375000,62.251501,72492
